In [1]:
# путь к JSON-файлу
json_file_path = '../data/result.json'
# путь к итоговому CSV-файлу
csv_file_path = '../data/comprehensive_output.csv'

### Load JSON

In [2]:
import json
import pandas as pd

try:
    # 1. Загрузка и чтение файла
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    messages_source_list = []
    if isinstance(data, dict) and 'messages' in data and isinstance(data['messages'], list):
        messages_source_list = data['messages']
        print(f"Обнаружен ключ 'messages' в JSON. Используется список сообщений из data['messages'] (всего {len(messages_source_list)} сообщений).")
    elif isinstance(data, list): # Если сам JSON является списком сообщений
        messages_source_list = data
        print(f"JSON является списком. Используется этот список как источник сообщений (всего {len(messages_source_list)} сообщений).")
    else:
        print("Структура JSON не соответствует ожидаемой (не найден ключ 'messages' со списком, или сам JSON не является списком).")
        print("Пожалуйста, проверьте структуру вашего JSON файла.")

    processed_data_for_df = []
    if messages_source_list:
        # 3. Обработка сообщений
        for message in messages_source_list:
            text_content = ""
            raw_text = message.get('text')

            if isinstance(raw_text, str):
                text_content = raw_text
            elif isinstance(raw_text, list):
                text_content = ''.join([part.get('text', '') if isinstance(part, dict) else str(part) for part in raw_text])
            
            # Заменяем символы новой строки на пробел (если нужно)
            text_content = text_content.replace('\n', ' ').replace('\r', ' ')

            # Извлекаем все поля из сообщения
            message_id = message.get('id')
            message_type = message.get('type') # Тип записи, обычно "message"
            date = message.get('date')
            date_unixtime = message.get('date_unixtime') # Unix timestamp
            sender_name = message.get('from') # Отображаемое имя отправителя
            sender_id = message.get('from_id') # ID отправителя (user_id, channel_id, etc.)
            reply_to_message_id = message.get('reply_to_message_id')
            forwarded_from_name = message.get('forwarded_from') # От кого переслано (имя)

            processed_data_for_df.append({
                'Message_ID': message_id,
                'Message_Type': message_type,
                'Date': date,
                'Date_Unixtime': date_unixtime,
                'Sender_Name': sender_name,
                'Sender_ID': sender_id,
                'Text': text_content,
                'Reply_To_Message_ID': reply_to_message_id,
                # 'Forwarded_From_Name': forwarded_from_name
                # Другие поля можно добавить по аналогии
            })
        
        df = pd.DataFrame(processed_data_for_df)
    else:
        df = pd.DataFrame() 

    if not df.empty:
        # Экспорт в CSV с использованием pandas
        # df.to_csv(csv_file_path, index=False, encoding='utf-8')
        # print(f"\nDataFrame успешно сохранен в CSV файл: {csv_file_path}")
        pass
    else:
        print("\nDataFrame пуст. CSV-файл не будет создан, так как не найдено сообщений для обработки.")

except FileNotFoundError:
    print(f"Ошибка: Файл {json_file_path} не найден.")
except json.JSONDecodeError:
    print(f"Ошибка: Не удалось декодировать JSON из файла {json_file_path}. Проверьте формат файла.")
except Exception as e:
    print(f"Произошла непредвиденная ошибка: {e}")

df.head()

Обнаружен ключ 'messages' в JSON. Используется список сообщений из data['messages'] (всего 1261 сообщений).


,Message_ID,Message_Type,Date,Date_Unixtime,Sender_Name,Sender_ID,Text,Reply_To_Message_ID
0,24907,message,2025-01-06T00:02:25,1736110945,Toheeb Bada,user7517994456,How can I withdraw my Ever,NaN
1,24908,message,2025-01-06T00:25:22,1736112322,David Mulish | I'll never DM first,user1835306285,Hello! If you want to withdraw EVER from your ...,24907.0
2,24917,message,2025-01-06T18:53:10,1736178790,German Limburger🐾,user5588232142,"Hey, everybody. Can anyone help me?",NaN
3,24918,message,2025-01-06T19:00:15,1736179215,David Mulish | I'll never DM first,user1835306285,"Hey German! You can ask me, here or in DM",24917.0
4,24919,message,2025-01-06T19:00:59,1736179259,David Mulish | I'll never DM first,user1835306285,"Be aware of scammers, admins never DM first!",NaN


### Находим админов

```python
{
    "@davidmuLish":"David Mulish | I'll never DM first",
    "@john_w_coLtrane": "John Coltrane (I'll Never DM First)",
    "@xestrum":"Vladislav Ponomarev (I'll never DM first)",
    "@lotumba":"Aleksandr Alekseev",
    "@eltcovai":"Andrew",
    "@Aleksandr_Everscale":"Aleksandr | Everscale"
}
```

In [3]:
admins = [
    "David Mulish | I'll never DM first",
    "John Coltrane (I'll Never DM First)",
    "Vladislav Ponomarev (I'll never DM first)",
    "Aleksandr Alekseev",
    "Andrew",
    "Aleksandr | Everscale"
]

admin_ids = df[df['Sender_Name'].isin(admins)]['Sender_ID'].unique()
admin_ids

array(['user1835306285', 'user5204190482', 'user183448583',
       'user37447050'], dtype=object)

In [4]:
df['is_admin'] = df['Sender_ID'].isin(admin_ids)
df.head()
# df.to_csv(csv_file_path, index=False, encoding='utf-8')
df.groupby('is_admin')['Message_ID'].count()

is_admin
False    779
True     482
Name: Message_ID, dtype: int64

In [5]:
df['Date'] = pd.to_datetime(df['Date'])
admin_sender_ids = ['user1835306285', 'user5204190482', 'user183448583','user37447050']

admin_replies = df[
    (df['Reply_To_Message_ID'].notna()) &
    (df['Sender_ID'].isin(admin_sender_ids))
].copy() # Используем .copy(), чтобы избежать SettingWithCopyWarning

admin_replies.head()

,Message_ID,Message_Type,Date,Date_Unixtime,Sender_Name,Sender_ID,Text,Reply_To_Message_ID,is_admin
1,24908,message,2025-01-06 00:25:22,1736112322,David Mulish | I'll never DM first,user1835306285,Hello! If you want to withdraw EVER from your ...,24907.0,True
3,24918,message,2025-01-06 19:00:15,1736179215,David Mulish | I'll never DM first,user1835306285,"Hey German! You can ask me, here or in DM",24917.0,True
8,24925,message,2025-01-06 20:05:08,1736183108,David Mulish | I'll never DM first,user1835306285,Hello! Have a good day too!,24922.0,True
9,24926,message,2025-01-06 20:05:16,1736183116,David Mulish | I'll never DM first,user1835306285,Exactly so,24923.0,True
12,24929,message,2025-01-06 20:14:01,1736183641,David Mulish | I'll never DM first,user1835306285,There are two ways: 1) you can withdraw EVER (...,24922.0,True


In [6]:
# Преобразуем Reply_To_Message_ID в целочисленный тип для объединения
admin_replies['Reply_To_Message_ID'] = admin_replies['Reply_To_Message_ID'].astype(int)

# Объединяем с исходным DataFrame, чтобы получить текст сообщения, на которое был дан ответ (вопроса)
answers_table = pd.merge(
    admin_replies,
    df[['Message_ID', 'Text']],
    left_on='Reply_To_Message_ID',
    right_on='Message_ID',
    how='left',
    suffixes=('_Answer', '_Question')
)

# Выбираем и переименовываем столбцы для итоговой таблицы
answers_table = answers_table[[
    'Sender_Name',
    'Sender_ID',
    'Date',
    'Text_Answer',
    'Reply_To_Message_ID',
    'Text_Question'
]]

answers_table.columns = [
    'Sender_Name',
    'Sender_ID',
    'Date',
    'Answer',
    'Reply_To_Message_ID',
    'Question'
]

# Форматируем столбец 'Date'
answers_table['Date'] = answers_table['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Выводим первые строки полученной таблицы
print(answers_table.head())

                          Sender_Name       Sender_ID                 Date  \
0  David Mulish | I'll never DM first  user1835306285  2025-01-06 00:25:22   
1  David Mulish | I'll never DM first  user1835306285  2025-01-06 19:00:15   
2  David Mulish | I'll never DM first  user1835306285  2025-01-06 20:05:08   
3  David Mulish | I'll never DM first  user1835306285  2025-01-06 20:05:16   
4  David Mulish | I'll never DM first  user1835306285  2025-01-06 20:14:01   

                                              Answer  Reply_To_Message_ID  \
0  Hello! If you want to withdraw EVER from your ...                24907   
1          Hey German! You can ask me, here or in DM                24917   
2                        Hello! Have a good day too!                24922   
3                                         Exactly so                24923   
4  There are two ways: 1) you can withdraw EVER (...                24922   

                                            Question  
0            

In [7]:
answers_table

,Sender_Name,Sender_ID,Date,Answer,Reply_To_Message_ID,Question
0,David Mulish | I'll never DM first,user1835306285,2025-01-06 00:25:22,Hello! If you want to withdraw EVER from your ...,24907,How can I withdraw my Ever
1,David Mulish | I'll never DM first,user1835306285,2025-01-06 19:00:15,"Hey German! You can ask me, here or in DM",24917,"Hey, everybody. Can anyone help me?"
2,David Mulish | I'll never DM first,user1835306285,2025-01-06 20:05:08,Hello! Have a good day too!,24922,Good day🙏 Tell me please how can I get Ever to...
3,David Mulish | I'll never DM first,user1835306285,2025-01-06 20:05:16,Exactly so,24923,Ever on Bybit - ERC20
4,David Mulish | I'll never DM first,user1835306285,2025-01-06 20:14:01,There are two ways: 1) you can withdraw EVER (...,24922,Good day🙏 Tell me please how can I get Ever to...
...,...,...,...,...,...,...
443,John Coltrane (I'll Never DM First),user5204190482,2025-05-17 19:26:28,"Hi. Write me in DM. Be careful, admins never w...",39300,Can someone helped me to get it
444,John Coltrane (I'll Never DM First),user5204190482,2025-05-18 02:40:28,Hello! If you have received a reward in Hamste...,39324,How to get my reward..
445,John Coltrane (I'll Never DM First),user5204190482,2025-05-18 03:01:01,"Write me in DM. Be careful, admins never write...",39326,pls help me with that
446,John Coltrane (I'll Never DM First),user5204190482,2025-05-19 17:41:37,"Hi, You need to import the seed phrase from To...",39408,I accidently transferred 10 tons though hamste...


In [8]:
answers_table.to_csv('../data/answers_table.csv', index=False, encoding='utf-8')

## DF for Chat History

In [9]:
df.head()

,Message_ID,Message_Type,Date,Date_Unixtime,Sender_Name,Sender_ID,Text,Reply_To_Message_ID,is_admin
0,24907,message,2025-01-06 00:02:25,1736110945,Toheeb Bada,user7517994456,How can I withdraw my Ever,NaN,False
1,24908,message,2025-01-06 00:25:22,1736112322,David Mulish | I'll never DM first,user1835306285,Hello! If you want to withdraw EVER from your ...,24907.0,True
2,24917,message,2025-01-06 18:53:10,1736178790,German Limburger🐾,user5588232142,"Hey, everybody. Can anyone help me?",NaN,False
3,24918,message,2025-01-06 19:00:15,1736179215,David Mulish | I'll never DM first,user1835306285,"Hey German! You can ask me, here or in DM",24917.0,True
4,24919,message,2025-01-06 19:00:59,1736179259,David Mulish | I'll never DM first,user1835306285,"Be aware of scammers, admins never DM first!",NaN,True


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1261 entries, 0 to 1260
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Message_ID           1261 non-null   int64         
 1   Message_Type         1261 non-null   object        
 2   Date                 1261 non-null   datetime64[ns]
 3   Date_Unixtime        1261 non-null   object        
 4   Sender_Name          1252 non-null   object        
 5   Sender_ID            1259 non-null   object        
 6   Text                 1261 non-null   object        
 7   Reply_To_Message_ID  639 non-null    float64       
 8   is_admin             1261 non-null   bool          
dtypes: bool(1), datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 80.2+ KB


In [11]:
def get_user_conversation_history(df: pd.DataFrame, target_sender_id: str, num_messages: int = 10) -> pd.DataFrame:
    """
    Возвращает последние N сообщений переписки с указанным пользователем,
    включая его собственные сообщения и ответы на них.

    Args:
        df (pd.DataFrame): Исходный DataFrame со всеми сообщениями.
        target_sender_id (str): Sender_ID пользователя, чью переписку нужно найти.
        num_messages (int): Количество последних сообщений для возврата.

    Returns:
        pd.DataFrame: DataFrame, содержащий отфильтрованные и отсортированные сообщения.
                      Включает столбец 'Question' для ответов.
    """

    # 1. Отфильтровываем все сообщения, которые отправил сам пользователь
    user_messages = df[df['Sender_ID'] == target_sender_id].copy()

    # 2. Находим сообщения, которые являются ответами на сообщения данного пользователя
    #    Для этого ищем Message_ID, которые были отправлены пользователем
    user_message_ids = user_messages['Message_ID'].tolist()
    
    # Находим все ответы, где Reply_To_Message_ID есть в списке Message_ID пользователя
    replies_to_user = df[
        df['Reply_To_Message_ID'].isin(user_message_ids)
    ].copy()

    # 3. Объединяем сообщения пользователя и ответы на них
    # Чтобы избежать дублирования, используем pd.concat и затем удаляем дубликаты по Message_ID
    # (хотя в данном случае дубликатов быть не должно, т.к. Message_ID уникален)
    conversation = pd.concat([user_messages, replies_to_user], ignore_index=True)
    
    # Удаляем дубликаты на всякий случай, если сообщение пользователя каким-то образом попало и в ответы (маловероятно)
    conversation.drop_duplicates(subset=['Message_ID'], inplace=True)

    # 4. Добавляем столбец с текстом вопроса (если это ответ)
    # Для сообщений, которые являются ответами (то есть имеют Reply_To_Message_ID),
    # находим исходный текст вопроса
    
    # Сначала преобразуем Reply_To_Message_ID в int для правильного мерджа
    # Обрабатываем NaN значения, чтобы они не вызывали ошибку при преобразовании
    replies_only = conversation[conversation['Reply_To_Message_ID'].notna()].copy()
    if not replies_only.empty:
        replies_only['Reply_To_Message_ID'] = replies_only['Reply_To_Message_ID'].astype(int)
        
        # Объединяем, чтобы получить текст исходного вопроса
        replies_with_questions = pd.merge(
            replies_only,
            df[['Message_ID', 'Text']],
            left_on='Reply_To_Message_ID',
            right_on='Message_ID',
            how='left',
            suffixes=('', '_Question') # суффикс для Text из df (т.е. вопроса)
        )
        
        # Переименовываем столбец с вопросом
        replies_with_questions.rename(columns={'Text_Question': 'Question_Text'}, inplace=True)
        
        # Объединяем обратно с основным набором, сохраняя только нужные колонки
        # Создаем пустой столбец 'Question_Text' во всей conversation, чтобы потом его заполнить
        conversation['Question_Text'] = None
        
        # Обновляем столбец 'Question_Text' только для тех строк, которые являются ответами
        # Используем index, чтобы правильно обновить значения
        conversation.set_index('Message_ID', inplace=True)
        replies_with_questions.set_index('Message_ID', inplace=True)
        
        conversation.update(replies_with_questions[['Question_Text']])
        conversation.reset_index(inplace=True) # Сброс индекса после update

    # 5. Сортируем по дате по убыванию и берем последние N сообщений
    conversation_sorted = conversation.sort_values(by='Date', ascending=False)
    
    # Выбираем только нужные колонки для вывода
    final_cols = [
        'Sender_Name', 'Sender_ID', 'Date', 'Text', 'Reply_To_Message_ID',
        'is_admin', 'Question_Text' # 'Question_Text' будет None для сообщений пользователя
    ]
    
    # Убедимся, что все колонки существуют перед выбором
    for col in ['Question_Text']:
        if col not in conversation_sorted.columns:
            conversation_sorted[col] = None # Добавим, если нет

    return conversation_sorted[final_cols].head(num_messages)

In [12]:
# 1. Показать последние 10 сообщений для 'user_a_id'
user_id_to_check = 'user1427686334'
conversation_history_user_a = get_user_conversation_history(df, user_id_to_check, num_messages=10)
print(f"--- Последние 10 сообщений для пользователя '{user_id_to_check}': ---")
print(conversation_history_user_a.to_string()) # .to_string() для полного вывода

print("\n" + "="*50 + "\n")

--- Последние 10 сообщений для пользователя 'user1427686334': ---
                            Sender_Name       Sender_ID                Date                                                                                               Text  Reply_To_Message_ID  is_admin                                                                                      Question_Text
11                         Арчи Маклауд  user1427686334 2025-05-07 22:14:59                                                                                      it works, thx              38616.0     False                                                        I have already topped up. Please try again.
16  John Coltrane (I'll Never DM First)  user5204190482 2025-05-07 22:10:27                                                        I have already topped up. Please try again.              38615.0      True                                   u mean i need to send 0.2 ever to address  form warming window ?
10                 

In [13]:
def format_conversation_as_dialog(conversation_df: pd.DataFrame, target_sender_id: str):
    """
    Форматирует DataFrame с историей переписки в читаемый диалог.

    Args:
        conversation_df (pd.DataFrame): DataFrame, полученный из get_user_conversation_history.
        target_sender_id (str): Sender_ID основного пользователя, чью переписку мы просматриваем.
    """
    
    # Сортируем по дате по возрастанию для правильного хронологического порядка диалога
    conversation_df_sorted = conversation_df.sort_values(by='Date', ascending=True)

    dialog_lines = []
    
    for index, row in conversation_df_sorted.iterrows():
        sender_name = row['Sender_Name']
        message_date = row['Date'].strftime('%Y-%m-%d %H:%M:%S')
        message_text = row['Text']
        question_text = row['Question_Text']
        
        # Определяем, является ли это сообщение целевым пользователем или ответом ему
        if row['Sender_ID'] == target_sender_id:
            # Сообщение от целевого пользователя
            dialog_lines.append(f"[{message_date}] **{sender_name} (USER):**")
            dialog_lines.append(f"  > {message_text}")
        else:
            # Сообщение от другого участника (вероятно, ответ)
            dialog_lines.append(f"[{message_date}] **{sender_name}:**")
            if pd.notna(question_text): # Если это ответ на конкретный вопрос
                dialog_lines.append(f"  На ваш вопрос: \"{question_text}\"")
                dialog_lines.append(f"  > {message_text}")
            else: # Если это просто сообщение от другого пользователя (не прямой ответ на наш вопрос)
                dialog_lines.append(f"  > {message_text}")
        
        dialog_lines.append("") # Пустая строка для разделения сообщений

    print("\n".join(dialog_lines))

In [14]:
user_id_to_check = 'user1580650390' # ID пользователя, для которого хотим получить диалог

# Получаем историю переписки
conversation_history = get_user_conversation_history(df, user_id_to_check, num_messages=10)

# Форматируем и выводим как диалог
print(f"--- Диалог с пользователем '{user_id_to_check}' (Последние 10 сообщений) ---")
format_conversation_as_dialog(conversation_history, user_id_to_check)

--- Диалог с пользователем 'user1580650390' (Последние 10 сообщений) ---
[2025-05-19 17:37:03] **Kyrie WalkerΣ (USER):**
  > I accidently transferred 10 tons though hamster network to tonkeeper ton wallet , can u please recover it?

[2025-05-19 17:41:37] **John Coltrane (I'll Never DM First):**
  На ваш вопрос: "I accidently transferred 10 tons though hamster network to tonkeeper ton wallet , can u please recover it?"
  > Hi, You need to import the seed phrase from Tonkeeper to Sparx.

[2025-05-19 17:51:06] **Kyrie WalkerΣ (USER):**
  > How

[2025-05-19 17:51:45] **Kyrie WalkerΣ (USER):**
  > I do have the transaction hash id included screenshot

[2025-05-19 17:52:04] **John Coltrane (I'll Never DM First):**
  На ваш вопрос: "How"
  > Write me a DM and I will explain everything to you.

